## LST Sensitivity calculation using pyirf

This notebook allow to calculate the best LST sensitivity using real data.
It uses MC gammas to optimize the best gammaness and $\theta^2$ cuts.
The gammaness cut can be optimized based on gamma efficiency or on the best sensitivity.
Cuts are then applied to ON/OFF data to calculate sensitivity.
Sensitivity using MC gammas as signal is also calculated.
Some useful plots such as rates, cuts per energy bin, angular resolution, etc. are shown at the end. 


### Needed imports

In [ ]:
from lstchain.io.io import read_dl2_to_pyirf, dl2_params_lstcam_key
import logging
import operator
import os
from lstchain.mc import plot_utils

import numpy as np
from astropy import table
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import QTable
import pandas as pd

from pyirf.io.eventdisplay import read_eventdisplay_fits
from pyirf.binning import (
    create_bins_per_decade,
    add_overflow_bins,
    create_histogram_table,
)
from pyirf.cuts import calculate_percentile_cut, evaluate_binned_cut
from pyirf.sensitivity import calculate_sensitivity, relative_sensitivity, estimate_background
from pyirf.utils import calculate_theta, calculate_source_fov_offset
from pyirf.benchmarks import energy_bias_resolution, angular_resolution

from pyirf.spectral import (
    calculate_event_weights,
    PowerLaw,
    CRAB_HEGRA,
    IRFDOC_PROTON_SPECTRUM
)
from pyirf.cut_optimization import optimize_gh_cut

from pyirf.irf import (
    effective_area_per_energy,
    energy_dispersion,
    psf_table,
    background_2d,
)

from pyirf.io import (
    create_aeff2d_hdu,
    create_psf_table_hdu,
    create_energy_dispersion_hdu,
    create_rad_max_hdu,
    create_background_2d_hdu,
)

log = logging.getLogger("pyirf")

### Initial variables

In [ ]:
#Observation time for sensitivity
T_OBS = 50 * u.hour
# scaling between on and off region.
# Make off region 5 times larger than on region for better
# background statistics
ALPHA = 1/5

# Radius to use for calculating bg rate
MAX_BG_RADIUS = 1.0 * u.deg

### Variables for pyirf cut optimisation
# gamma efficiency used for first calculation of the binned theta cuts
# initial theta cuts are calculated using a fixed g/h cut corresponding to this efficiency
# then g/h cuts are optimized after applying these initial theta cuts. 
INITIAL_GH_CUT_EFFICENCY = 0.5
#gamma efficiency used for gh cuts calculation
MAX_GH_CUT_EFFICIENCY = 0.8
GH_CUT_EFFICIENCY_STEP = 0.01

### Variables for 2d grid theta/gammaness cut optimisation
THETA_CUT_MIN = 0.0125
THETA_CUT_MAX = 1
THETA_CUT_STEP = 0.0125
GH_CUT_MIN = 0
GH_CUT_MAX = 1
GH_CUT_STEP = 0.0125

#Number of energy bins
N_EBINS = 15
#Energy range
EMIN=0.01  #TeV
EMAX=50  #TeV

#Fixed cuts
INTENSITY_CUT = 200
LEAKAGE2_CUT = 0.2

### Read hdf5 files into pyirf format

In [ ]:
path_to_data = '/fefs/aswg/workspace/david.miranda/data/prod5/dl2/lstchain_v.0.6.3/mono-lst-sipm-pmma-3ns/50.50/intensity_200_leakage_0.2'

file_gammas = os.path.join(path_to_data, 'dl2_gamma_on_merge_test.h5')
file_protons = os.path.join(path_to_data, 'dl2_proton_merge_test.h5')

#file_on = "/home/queenmab/DATA/LST/ON/Run2007_on.h5"
#file_off = "/home/queenmab/DATA/LST/OFF.h5"

In [ ]:
events_g, simu_info_g = read_dl2_to_pyirf(file_gammas)
events_p, simu_info_p = read_dl2_to_pyirf(file_protons)

### Apply quality cuts

In [ ]:
for events in (events_g, events_p):#, events_on, events_off):
        events['good_events'] = (events['intensity']>=INTENSITY_CUT) & \
        (events['leakage_intensity_width_2']<=LEAKAGE2_CUT) & \
        (events['tel_id'] == 1.0)

In [ ]:
particles = {
         "gamma": {
                "events": events_g[events_g['good_events']],
                "simulation_info": simu_info_g,
                "target_spectrum": CRAB_HEGRA
            },
        "proton": {
                "events": events_p[events_p['good_events']],
                "simulation_info": simu_info_p,
                "target_spectrum": IRFDOC_PROTON_SPECTRUM
            },
        #"off": {
        #        "events": events_off[events_off['good_events']]
        #    },
        #"on":{
        #        "events": events_on[events_on['good_events']]
        #    }
        }

In [ ]:
# Manage MC gammas:
#Get simulated spectrum
particles["gamma"]["simulated_spectrum"] = PowerLaw.from_simulation(particles["gamma"]["simulation_info"], 
                                                                    T_OBS)
#Reweight to target spectrum (Crab Hegra)
particles["gamma"]["events"]["weight"] = calculate_event_weights(
        particles["gamma"]["events"]["true_energy"], 
        particles["gamma"]["target_spectrum"], 
        particles["gamma"]["simulated_spectrum"]
        )
for prefix in ('true', 'reco'):
    k = f"{prefix}_source_fov_offset"
    particles["gamma"]["events"][k] = calculate_source_fov_offset(particles["gamma"]["events"], prefix=prefix) 
particles["gamma"]["events"]["source_fov_offset"] = calculate_source_fov_offset(particles["gamma"]["events"])
# calculate theta / distance between reco and assumed source position
# we handle only ON observations here, so the assumed source position
# is the pointing position
particles["gamma"]["events"]["theta"] = calculate_theta(
        particles["gamma"]["events"],
        assumed_source_az=particles["gamma"]["events"]["true_az"],
        assumed_source_alt=particles["gamma"]["events"]["true_alt"],
        )

In [ ]:
# Manage MC protons:
#Get simulated spectrum
particles["proton"]["simulated_spectrum"] = PowerLaw.from_simulation(particles["proton"]["simulation_info"], 
                                                                    T_OBS)
#Reweight to target spectrum:
particles["proton"]["events"]["weight"] = calculate_event_weights(
        particles["proton"]["events"]["true_energy"], 
        particles["proton"]["target_spectrum"], 
        particles["proton"]["simulated_spectrum"]
        )
for prefix in ('true', 'reco'):
    k = f"{prefix}_source_fov_offset"
    particles["proton"]["events"][k] = calculate_source_fov_offset(particles["proton"]["events"], prefix=prefix)        
# calculate theta / distance between reco and assumed source position
# we handle only ON observations here, so the assumed source position
# is the pointing position
particles["proton"]["events"]["theta"] = calculate_theta(
        particles["proton"]["events"],
        assumed_source_az=particles["proton"]["events"]["pointing_az"],
        assumed_source_alt=particles["proton"]["events"]["pointing_alt"],
        )

In [ ]:
#Get events
gammas = particles["gamma"]["events"]
protons = particles["proton"]["events"]

## Calculate the best cuts for sensitivity

### Define bins

In [ ]:
#Sensitivity energy bins
sensitivity_bins = np.logspace(np.log10(EMIN),
    np.log10(EMAX), N_EBINS + 1) * u.TeV

### Data to optimize best cuts

In [ ]:
signal=gammas
background=protons

## Method : pyirf opt_gh_cut

In [ ]:
"""
Only one cut optimisation method shoud be used. Method 1
"""

#Calculate an initial GH cut for calculating initial theta cuts, based on INITIAL_GH_CUT_EFFICIENCY
INITIAL_GH_CUT = np.quantile(signal['gh_score'], (1 - INITIAL_GH_CUT_EFFICENCY))
INITIAL_GH_CUT

### Initial $\theta$ cut

In [ ]:
# theta cut is 68 percent containmente of the gammas
# for now with a fixed global, unoptimized score cut
mask_theta_cuts = signal["gh_score"] >= INITIAL_GH_CUT
theta_cuts = calculate_percentile_cut(
        signal["theta"][mask_theta_cuts],
        signal["reco_energy"][mask_theta_cuts],
        bins=sensitivity_bins,
        fill_value=np.nan * u.deg,
        percentile=68,
)

In [ ]:
# evaluate the initial theta cut
signal["selected_theta"] = evaluate_binned_cut(
    signal["theta"], signal["reco_energy"], theta_cuts, operator.le
    )

### Run block below for G/H cut optimization based on best sensitivity

In [ ]:
log.info("Optimizing G/H separation cut for best sensitivity")
gh_cut_efficiencies = np.arange(
        GH_CUT_EFFICIENCY_STEP,
        MAX_GH_CUT_EFFICIENCY + GH_CUT_EFFICIENCY_STEP / 2,
        GH_CUT_EFFICIENCY_STEP
)
sensitivity_step_2, gh_cuts = optimize_gh_cut(
    signal[signal["selected_theta"]],
    background,
    reco_energy_bins=sensitivity_bins,
    gh_cut_efficiencies=gh_cut_efficiencies,
    theta_cuts=theta_cuts,
    op=operator.ge,
    alpha=ALPHA,
    background_radius=MAX_BG_RADIUS,
)

### Setting of $\theta$ cut as 68% containment of events surviving the cuts

In [ ]:
#Evaluate gh cut:
for tab in (gammas, protons):#, on, off):
    tab["selected_gh"] = evaluate_binned_cut(
    tab["gh_score"], tab["reco_energy"], gh_cuts, operator.ge
    )

theta_cuts_opt = calculate_percentile_cut(
    gammas[gammas["selected_gh"]]["theta"],
    gammas[gammas["selected_gh"]]["reco_energy"],
    sensitivity_bins,
    percentile=68,
    fill_value=0.32 * u.deg,
    )

"""
End of Method 1
"""

## Method : Alternative grid optimisation theta/gh cuts

In [ ]:
"""
Only one cut optimisation method shoud be used. Method 2
"""

log.info("Optimizing theta cut and G/H separation cut for best sensitivity")
thetacuts = np.arange(THETA_CUT_MIN,THETA_CUT_MAX,THETA_CUT_STEP)
ghcuts = np.arange(GH_CUT_MIN,GH_CUT_MAX,GH_CUT_STEP)
opt_theta_cut = []
opt_gh_cut = []
opt_sensi = []
for i, ebins in enumerate(sensitivity_bins[:-1]):
    sensitivity_step_2 = np.ones(shape=(len(thetacuts),len(ghcuts)))
    for j, tcut in enumerate(thetacuts):
        for k, ghcut in enumerate(ghcuts):
            non = np.sum(gammas['weight'][((gammas["reco_energy"]>sensitivity_bins[i])&(gammas["reco_energy"]<sensitivity_bins[i+1])&
                   (gammas["theta"].value<tcut) & (gammas["gh_score"]>ghcut))])
            noff = np.sum(protons['weight'][((protons["reco_energy"]>sensitivity_bins[i])&(protons["reco_energy"]<sensitivity_bins[i+1])&
                    (protons["theta"]<MAX_BG_RADIUS) & (protons["gh_score"]>ghcut))]) * (tcut/MAX_BG_RADIUS.value)**2
            if noff == 0:
                non = 0
            alpha = ALPHA
            non = non + alpha * noff
            sensitivity_step_2[j][k] = relative_sensitivity(
                n_on=non,
                n_off=noff,
                alpha=alpha,
                min_significance=5,
                min_signal_events=10,
                min_excess_over_background=0.05
            )
    if not np.all(np.isnan(sensitivity_step_2)):
        # nanargmin won't return the index of nan entries
        jmax,kmax = np.unravel_index(np.nanargmin(sensitivity_step_2), sensitivity_step_2.shape)
        opt_theta_cut.append(thetacuts[jmax])
        opt_gh_cut.append(ghcuts[kmax])
        opt_sensi.append(sensitivity_step_2[jmax][kmax])
    else:
        opt_theta_cut.append(np.inf)
        opt_gh_cut.append(np.inf)
        opt_sensi.append(np.inf)
    print(i,end=', ')

In [ ]:
from astropy.table import QTable
from pyirf.binning import  bin_center

In [ ]:
gh_cuts = QTable()
gh_cuts["low"] = sensitivity_bins[0:-1]
gh_cuts["center"] = bin_center(sensitivity_bins)
gh_cuts["high"] = sensitivity_bins[1:]
gh_cuts["cut"] = opt_gh_cut

theta_cuts_opt = QTable()
theta_cuts_opt["low"] = sensitivity_bins[0:-1]
theta_cuts_opt["center"] = bin_center(sensitivity_bins)
theta_cuts_opt["high"] = sensitivity_bins[1:]
theta_cuts_opt["cut"] = opt_theta_cut* u.deg


"""
End of Method 2
"""

### Evaluate optimized cuts

In [ ]:
#Evaluate gh cut
for tab in (gammas, protons):#, on, off):
    tab["selected_gh"] = evaluate_binned_cut(
    tab["gh_score"], tab["reco_energy"], gh_cuts, operator.ge
    )

In [ ]:
#Evaluate theta cut
gammas["selected_theta"] = evaluate_binned_cut(
        gammas["theta"], gammas["reco_energy"], theta_cuts_opt, operator.le
    )
gammas["selected"] = gammas["selected_theta"] & gammas["selected_gh"]

protons["selected"] =  protons["selected_gh"]

### Create event histograms

In [ ]:
gamma_hist = create_histogram_table(
    gammas[gammas["selected"]], bins=sensitivity_bins
    )    
proton_hist = estimate_background(
    protons[protons["selected"]],
    reco_energy_bins=sensitivity_bins,
    theta_cuts=theta_cuts_opt,
    alpha=ALPHA,
    background_radius=MAX_BG_RADIUS,
    )

### Sensitivity with MC gammas and protons

In [ ]:
sensitivity_mc = calculate_sensitivity(
    gamma_hist, proton_hist, alpha=ALPHA
)

In [ ]:
# Compute errors on the sensitivity based on the number of simulated gamma and protons selected
gamma_hist_errl = QTable()
gamma_hist_errl["reco_energy_low"] = gamma_hist["reco_energy_low"]
gamma_hist_errl["reco_energy_high"] = gamma_hist["reco_energy_high"]
gamma_hist_errl["reco_energy_center"] = gamma_hist["reco_energy_center"]
gamma_hist_errh = QTable()
gamma_hist_errh["reco_energy_low"] = gamma_hist["reco_energy_low"]
gamma_hist_errh["reco_energy_high"] = gamma_hist["reco_energy_high"]
gamma_hist_errh["reco_energy_center"] = gamma_hist["reco_energy_center"]
weights = gamma_hist["n_weighted"].value/gamma_hist["n"]
errn=np.sqrt(gamma_hist["n"])
errn_weighted = errn * weights
gamma_hist_errl["n"] = gamma_hist["n"] - errn
gamma_hist_errl["n_weighted"] = gamma_hist["n_weighted"] - errn_weighted
gamma_hist_errh["n"] = gamma_hist["n"] + errn
gamma_hist_errh["n_weighted"] = gamma_hist["n_weighted"] + errn_weighted

proton_hist_errl = QTable()
proton_hist_errl["reco_energy_low"] = proton_hist["reco_energy_low"]
proton_hist_errl["reco_energy_high"] = proton_hist["reco_energy_high"]
proton_hist_errl["reco_energy_center"] = proton_hist["reco_energy_center"]
proton_hist_errh = QTable()
proton_hist_errh["reco_energy_low"] = proton_hist["reco_energy_low"]
proton_hist_errh["reco_energy_high"] = proton_hist["reco_energy_high"]
proton_hist_errh["reco_energy_center"] = proton_hist["reco_energy_center"]
weights = proton_hist["n_weighted"].value/proton_hist["n"]
errn=np.sqrt(proton_hist["n"]/ALPHA) * theta_cuts_opt["cut"].value/MAX_BG_RADIUS.value
errn_weighted = errn * weights
proton_hist_errl["n"] = proton_hist["n"] - errn
proton_hist_errl["n_weighted"] = proton_hist["n_weighted"] - errn_weighted
proton_hist_errh["n"] = proton_hist["n"] + errn
proton_hist_errh["n_weighted"] = proton_hist["n_weighted"] + errn_weighted
proton_hist_errl["n_weighted"][proton_hist_errl["n_weighted"]<0]=0
gamma_hist_errl["n_weighted"][gamma_hist_errl["n_weighted"]<0]=0
sensitivity_mc_l = calculate_sensitivity(
    gamma_hist_errl, proton_hist_errh, alpha=ALPHA
)
sensitivity_mc_h = calculate_sensitivity(
    gamma_hist_errh, proton_hist_errl, alpha=ALPHA
)
sensitivity_mc["relative_sensitivity_errl"]=(sensitivity_mc_l["relative_sensitivity"]-sensitivity_mc["relative_sensitivity"])
sensitivity_mc["relative_sensitivity_errh"]=(sensitivity_mc["relative_sensitivity"]-sensitivity_mc_h["relative_sensitivity"])

In [ ]:
#add cuts in table
sensitivity_mc["gh_cut"]=gh_cuts["cut"]

In [ ]:
sensitivity_mc

### Plot Sensitivity curves

In [ ]:
# scale relative sensitivity by Crab flux to get the flux sensitivity
spectrum = particles['gamma']['target_spectrum']
sensitivity_mc["flux_sensitivity"] = (
        sensitivity_mc["relative_sensitivity"] * spectrum(sensitivity_mc["reco_energy_center"])
     )

In [ ]:
sensitivity_mc["flux_sensitivity_errl"] = (
        sensitivity_mc["relative_sensitivity_errl"] * spectrum(sensitivity_mc["reco_energy_center"])
     )
sensitivity_mc["flux_sensitivity_errh"] = (
        sensitivity_mc["relative_sensitivity_errh"] * spectrum(sensitivity_mc["reco_energy_center"])
     )

In [ ]:
sensitivity_mc_saved=QTable.read('sensitivity_lhfit_pyirf_gridcut_errors.hdf5')

In [ ]:
#Optional copy of processed object for multi-results plots
sensitivity_mc_saved=sensitivity_mc
theta_cuts_saved = theta_cuts_opt
gh_cuts_saved=gh_cuts

In [ ]:
sensitivity_mc_saved

In [ ]:
#Optional copy of processed object for multi-results plots
sensitivity_mc_saved_optghcut=sensitivity_mc
theta_cuts_saved_optghcut = theta_cuts_opt
gh_cuts_saved_optghcut=gh_cuts

In [ ]:
#Optionnal line to write the results to file
sensitivity_mc.write('sensitivity_lhfit_pyirf_gridcut_errors.hdf5',serialize_meta=True)

In [ ]:
"""
e0,s0 = np.genfromtxt("/fefs/aswg/workspace/gabriel.emery/sensitivity_lstchain.txt").T
e0 = 1000*e0
s0=s0/10000

el1,eh1,s1,serr1,ct1,cg1 = np.genfromtxt(
    "/fefs/aswg/workspace/gabriel.emery/DL1/CyrilCode/size_cut_100_wol_0.01_1_leakage0_0.2/sensitivity/sensitivity.txt").T
e1 = (el1+eh1)/2
e1 = np.power(10,e1)
el1 = np.power(10,el1)
eh1 = np.power(10,eh1)
errl1=e1-el1
errh1=eh1-e1

s1=s1/10000
serr1 = serr1/10000
"""

plt.figure(figsize=(12,8))
ax=plt.axes()
unit = u.Unit('TeV cm-2 s-1')
"""
e = sensitivity_mc_saved_optghcut['reco_energy_center']

s_mc = (e**2 * sensitivity_mc_saved_optghcut['flux_sensitivity'])
s_mc_errl = (e**2 * sensitivity_mc_saved_optghcut['flux_sensitivity_errl'])
s_mc_errh = (e**2 * sensitivity_mc_saved_optghcut['flux_sensitivity_errh'])

plt.errorbar(
    e.to_value(u.GeV),
    s_mc.to_value(unit),
    xerr=(sensitivity_mc_saved_optghcut['reco_energy_high'] - sensitivity_mc_saved_optghcut['reco_energy_low']).to_value(u.GeV) / 2,
    yerr=[s_mc_errl.to_value(unit),s_mc_errh.to_value(unit)],
    label='LHfit pyirf.opt_gh_cut intensity cut 100 leak cut 0.2',
    #label='MC gammas/protons',
    fmt =' ', color = 'b'
    )
"""
e = sensitivity_mc['reco_energy_center']

s_mc = (e**2 * sensitivity_mc['flux_sensitivity'])
s_mc_errl = (e**2 * sensitivity_mc['flux_sensitivity_errl'])
s_mc_errh = (e**2 * sensitivity_mc['flux_sensitivity_errh'])

plt.errorbar(
    e.to_value(u.GeV),
    s_mc.to_value(unit),
    xerr=(sensitivity_mc['reco_energy_high'] - sensitivity_mc['reco_energy_low']).to_value(u.GeV) / 2,
    yerr=[s_mc_errl.to_value(unit),s_mc_errh.to_value(unit)],
    label='LHfit gamma gridcut noff!=0 intensity cut 200 leak cut 0.2',
    #label='MC gammas/protons',
    fmt =' ', color = 'k'
    )

e = sensitivity_mc_saved['reco_energy_center']

s_mc = (e**2 * sensitivity_mc_saved['flux_sensitivity'])
s_mc_errl = (e**2 * sensitivity_mc_saved['flux_sensitivity_errl'])
s_mc_errh = (e**2 * sensitivity_mc_saved['flux_sensitivity_errh'])

plt.errorbar(
    e.to_value(u.GeV),
    s_mc.to_value(unit),
    xerr=(sensitivity_mc_saved['reco_energy_high'] - sensitivity_mc_saved['reco_energy_low']).to_value(u.GeV) / 2,
    yerr=[s_mc_errl.to_value(unit),s_mc_errh.to_value(unit)],
    label='LHfit gamma gridcut noff!=0 intensity cut 100 leak cut 0.2',
    #label='MC gammas/protons',
    fmt =' ', color = 'c'
    )



#plt.errorbar(x=e0,y=s0,fmt='.',label="Prod5 lstchain size cut 50", color = 'b')
#plt.errorbar(x=e1*1000,xerr=[errl1*1000,errh1*1000],y=e1*e1*s1,
#             yerr=e1*e1*serr1,fmt='.',label="Prod5 digicampipe LST1234 LHfit size cut 100 leak cut 0.2 ", color = 'g')

#Plot magic sensitivity
s = np.loadtxt('/home/david.miranda/software/cta-lstchain_v.0.6.3/lstchain/spectra/data/magic_sensitivity.txt', skiprows = 1)
ax.loglog(s[:,0], s[:,3] * np.power(s[:,0]/ 1e3, 2),
              color = 'black', label = 'MAGIC (Aleksic et al. 2014)')

#Plot Crab SED
plot_utils.plot_Crab_SED(ax, 100, 5*u.GeV, 1e4*u.GeV, label="100% Crab") #Energy in GeV
plot_utils.plot_Crab_SED(ax, 10, 5*u.GeV, 1e4*u.GeV, linestyle='--', label="10% Crab") #Energy in GeV
plot_utils.plot_Crab_SED(ax, 1, 5*u.GeV, 1e4*u.GeV, linestyle=':', label="1% Crab") #Energy in GeV


# Style settings
plt.title('Minimal Flux Needed for 5σ Detection in 50 hours')
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Reconstructed energy [GeV]")
plt.ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")
plt.grid(which="both")
plt.legend()
plt.show()

In [ ]:
#ratio plot between two sensitivities
plt.errorbar(e.to_value(u.GeV),sensitivity_mc['flux_sensitivity']/sensitivity_mc_saved['flux_sensitivity'],
    xerr=(sensitivity_mc_saved['reco_energy_high'] - sensitivity_mc_saved['reco_energy_low']).to_value(u.GeV) / 2,fmt= ".")
plt.xscale("log")
plt.show()

### Rates

In [ ]:
rate_gammas = gamma_hist["n_weighted"]/T_OBS.to(u.min)
area_ratio_p = (1-np.cos(theta_cuts_opt['cut']))/(1-np.cos(MAX_BG_RADIUS))
rate_proton = proton_hist["n_weighted"]*area_ratio_p/T_OBS.to(u.min)

plt.errorbar(
    0.5 * (gamma_hist['reco_energy_low'] + gamma_hist['reco_energy_high']).to_value(u.TeV),
    rate_gammas.to_value(1/u.min),
    xerr=0.5 * (gamma_hist['reco_energy_high'] - gamma_hist['reco_energy_low']).to_value(u.TeV),
    label='Gammas MC',
)
plt.errorbar(
    0.5 * (proton_hist['reco_energy_low'] + proton_hist['reco_energy_high']).to_value(u.TeV),
    rate_proton.to_value(1/u.min),
    xerr=0.5 * (proton_hist['reco_energy_high'] - proton_hist['reco_energy_low']).to_value(u.TeV),
    label='Protons MC',
)
plt.legend()
plt.ylabel('Rate events/min')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.yscale('log')
plt.grid(which="both")
plt.show()

### Cuts

In [ ]:
"""plt.errorbar(
    0.5 * (theta_cuts_saved_optghcut['low'] + theta_cuts_saved_optghcut['high']).to_value(u.TeV),
    (theta_cuts_saved_optghcut['cut']**2).to_value(u.deg**2),
    xerr=0.5 * (theta_cuts_saved_optghcut['high'] - theta_cuts_saved_optghcut['low']).to_value(u.TeV),
    ls='',
    color='b'
    )
"""
plt.errorbar(
    0.5 * (theta_cuts_saved['low'] + theta_cuts_saved['high']).to_value(u.TeV),
    (theta_cuts_saved['cut']**2).to_value(u.deg**2),
    xerr=0.5 * (theta_cuts_saved['high'] - theta_cuts_saved['low']).to_value(u.TeV),
    ls='',
    color='c'
    )
plt.errorbar(
    0.5 * (theta_cuts_opt['low'] + theta_cuts_opt['high']).to_value(u.TeV),
    (theta_cuts_opt['cut']**2).to_value(u.deg**2),
    xerr=0.5 * (theta_cuts_opt['high'] - theta_cuts_opt['low']).to_value(u.TeV),
    ls='',
    color='k'
    )
plt.ylabel(r'$\theta^2$-cut')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.grid(which="both")
plt.show()

In [ ]:
"""plt.errorbar(
    0.5 * (gh_cuts_saved_optghcut['low'] + gh_cuts_saved_optghcut['high']).to_value(u.TeV),
    gh_cuts_saved_optghcut['cut'],
    xerr=0.5 * (gh_cuts_saved_optghcut['high'] - gh_cuts_saved_optghcut['low']).to_value(u.TeV),
    ls='',
    color='b'
    )"""
plt.errorbar(
    0.5 * (gh_cuts_saved['low'] + gh_cuts_saved['high']).to_value(u.TeV),
    gh_cuts_saved['cut'],
    xerr=0.5 * (gh_cuts_saved['high'] - gh_cuts_saved['low']).to_value(u.TeV),
    ls='',
    color='c'
    )
plt.errorbar(
    0.5 * (gh_cuts['low'] + gh_cuts['high']).to_value(u.TeV),
    gh_cuts['cut'],
    xerr=0.5 * (gh_cuts['high'] - gh_cuts['low']).to_value(u.TeV),
    ls='',
    color='k'
    )
plt.ylabel('G/H-cut')
plt.xlabel(r'$E_\mathrm{reco} / \mathrm{TeV}$')
plt.xscale('log')
plt.grid(which="both")
plt.show()

### Angular Resolution

In [ ]:
selected_events_gh = table.vstack(
        gammas[gammas["selected_gh"]], protons[protons["selected_gh"]])

In [ ]:
#Optional copy of processed object for multi-results plots
selected_events_gh_saved = selected_events_gh

In [ ]:
#Optional copy of processed object for multi-results plots
selected_events_gh_saved_optghcut = selected_events_gh

In [ ]:
"""
ang_res = angular_resolution(selected_events_gh_saved_optghcut[selected_events_gh_saved_optghcut["selected_gh"]], sensitivity_bins,)
plt.errorbar(
    0.5 * (ang_res['true_energy_low'] + ang_res['true_energy_high']),
    ang_res['angular_resolution'],
    xerr=0.5 * (ang_res['true_energy_high'] - ang_res['true_energy_low']),
    ls='',color='b',
    label='LHfit pyirf.opt_gh_cut intensity cut 100 leak cut 0.2'
    )
"""
ang_res = angular_resolution(selected_events_gh_saved[selected_events_gh_saved["selected_gh"]], sensitivity_bins,)
plt.errorbar(
    0.5 * (ang_res['true_energy_low'] + ang_res['true_energy_high']),
    ang_res['angular_resolution'],
    xerr=0.5 * (ang_res['true_energy_high'] - ang_res['true_energy_low']),
    ls='',color='c',
    label='LHfit gridcut noff!=0 intensity cut 100 leak cut 0.2'
    )
ang_res = angular_resolution(selected_events_gh[selected_events_gh["selected_gh"]], sensitivity_bins,)
plt.errorbar(
    0.5 * (ang_res['true_energy_low'] + ang_res['true_energy_high']),
    ang_res['angular_resolution'],
    xerr=0.5 * (ang_res['true_energy_high'] - ang_res['true_energy_low']),
    ls='',color='r',
    label='Std gridcut noff!=0 intensity cut 100 leak cut 0.2'
    )
# Style settings
plt.xlim(1.e-2, 2.e2)
plt.ylim(0.5e-1, 1.2)
plt.xscale("log")
#plt.yscale("log")
plt.xlabel("True energy / TeV")
plt.ylabel("Angular Resolution / deg")
plt.grid(which="both")
plt.legend()
plt.show()

### Energy resolution

In [ ]:
selected_events = table.vstack(
        gammas[gammas["selected"]], protons[protons["selected"]])

In [ ]:
#Optional copy of processed object for multi-results plots
selected_events_saved = selected_events
bias_resolution_saved = energy_bias_resolution(
    selected_events, sensitivity_bins,
        )

In [ ]:
#Optional copy of processed object for multi-results plots
selected_events_saved_optghcut = selected_events
bias_resolution_saved_optghcut = energy_bias_resolution(
    selected_events, sensitivity_bins,
        )

In [ ]:
selected_events = table.vstack(
        gammas[gammas["selected"]], protons[protons["selected"]])

bias_resolution = energy_bias_resolution(
    selected_events, sensitivity_bins,
        )

# Plot function
"""
plt.errorbar(
    0.5 * (bias_resolution_saved_optghcut['true_energy_low'] + bias_resolution_saved_optghcut['true_energy_high']),
    bias_resolution_saved_optghcut['resolution'],
    xerr=0.5 * (bias_resolution_saved_optghcut['true_energy_high'] - bias_resolution_saved_optghcut['true_energy_low']),
    ls='',color='b',
    label='LHfit pyirf.opt_gh_cut intensity cut 100 leak cut 0.2'
    )
"""
plt.errorbar(
    0.5 * (bias_resolution_saved['true_energy_low'] + bias_resolution_saved['true_energy_high']),
    bias_resolution_saved['resolution'],
    xerr=0.5 * (bias_resolution_saved['true_energy_high'] - bias_resolution_saved['true_energy_low']),
    ls='',color='c',
    label='LHfit gridcut noff!=0 intensity cut 100 leak cut 0.2'
    )
plt.errorbar(
    0.5 * (bias_resolution['true_energy_low'] + bias_resolution['true_energy_high']),
    bias_resolution['resolution'],
    xerr=0.5 * (bias_resolution['true_energy_high'] - bias_resolution['true_energy_low']),
    ls='',color='r',
    label='Std gridcut noff!=0 intensity cut 100 leak cut 0.2'
    )
plt.xscale('log')

# Style settings
plt.xlabel(r"$E_\mathrm{True} / \mathrm{TeV}$")
plt.ylabel("Energy resolution")
plt.grid(which="both")
plt.legend(loc="best")
plt.show()

### Reco Alt/Az for MC selected events

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(30,4))
emin_bins=[0.0, 0.1,0.5, 1, 5] * u.TeV
emax_bins=[0.1,0.5,1,5,10] * u.TeV
selected_events=gammas[gammas["selected"]]
for i, ax in enumerate(axs):
    events=selected_events[(selected_events['reco_energy']>emin_bins[i]) & \
                                    (selected_events['reco_energy']<emax_bins[i]) ]
    pcm = ax.hist2d(events['reco_az'].to_value(u.deg), events['reco_alt'].to_value(u.deg), bins=50)
    ax.title.set_text("%.1f-%.1f TeV" % (emin_bins[i].to_value(), emax_bins[i].to_value()))
    ax.set_xlabel("Az (º)")
    ax.set_ylabel("Alt (º)")
    fig.colorbar(pcm[3], ax=ax)
plt.show()

### Checks on number of islands

In [ ]:
gammas_selected=gammas[(gammas['selected']) & (gammas['reco_energy']>1.*u.TeV)]
plt.hist(gammas_selected['n_islands'], bins=10, range=(0.5, 10.5))
plt.yscale('log')

In [ ]:
protons_selected=protons[(protons['selected']) & (protons['reco_energy']>1.*u.TeV)]
plt.hist(protons_selected['n_islands'], bins=10, range=(0.5, 10.5))
plt.yscale('log')

### Gammaness histograms

In [ ]:
gammas_selected=gammas[(gammas['reco_energy']>sensitivity_bins[0])& (gammas['reco_energy']<sensitivity_bins[N_EBINS])]
protons_selected=protons[(protons['reco_energy']>sensitivity_bins[0])& (protons['reco_energy']<sensitivity_bins[N_EBINS])]
plt.hist(gammas_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5)
plt.hist(protons_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5)
plt.yscale('log')

In [ ]:
i=13
gammas_selected=gammas[(gammas['reco_energy']>sensitivity_bins[i])& (gammas['reco_energy']<sensitivity_bins[i+1])]
protons_selected=protons[(protons['reco_energy']>sensitivity_bins[i])& (protons['reco_energy']<sensitivity_bins[i+1])]
plt.hist(gammas_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5,color='c')
plt.hist(protons_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5,color='k')
gammas_selected=gammas[(gammas['selected_theta']) & (gammas['reco_energy']>sensitivity_bins[i])& (gammas['reco_energy']<sensitivity_bins[i+1])]
protons_selected=protons[(protons['theta'].value< 1) & (protons['reco_energy']>sensitivity_bins[i])& (protons['reco_energy']<sensitivity_bins[i+1])]
plt.hist(gammas_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5,color='c')
plt.hist(protons_selected['gh_score'], bins=100, range=(0, 1),alpha=0.5,color='k')
plt.title("gammaness Erange "+str(sensitivity_bins[i])+"--"+str(sensitivity_bins[i+1]))
plt.vlines(gh_cuts['cut'][i],ymin=plt.ylim()[0],ymax=plt.ylim()[1], color = 'r', label = "gh_cut")
plt.legend()
plt.yscale('log')